In [ ]:
# Image name
#imname = 'ISIC_2454082' # easy nevus
#imname = 'ISIC_0495948' # hard nevus
imname = 'ISIC_7889097' # easy melanoma
#imname = 'ISIC_7949783' # hard melanoma
#imname = 'ISIC_6736963' # single case
#imname = '20161227134727875'
#imname = '20170328153241782'
#imname = '20201027154530702'
imname = '1604950323_3962119'

# display type
distype = 'logodds'

# classes
target = 'melanoma'
logodds = 'melanoma~other'

# figure size
figsize = (12,12)

In [ ]:
# imports
import glob

from isicarchive import imfunc
import matplotlib.pyplot as plt
import numpy

def scaledimshow(im, ax=None):
    if ax is None:
        f = plt.figure(figsize=figsize)
    im = im.copy()
    am = numpy.max(numpy.abs(im))
    if numpy.any(im < 0.0):
        print([-am,am])
        im[0,0] = -am
    else:
        print([0.0,am])
    im[-1,-1] = am
    return plt.imshow(im)

In [ ]:
# load maps for one image and type/class
pstd = None
if distype[0] == 'h':
    try:
        heatmaps = numpy.load(glob.glob('heatmaps' + '/' + imname + '/*' + target + '_*mean.npz')[0])
        pmean = heatmaps['arr_0'].copy()
        pstd = heatmaps['arr_1'].copy()
    except:
        print(f'No heatmap for {imname}')
if distype[0] == 'l':
    try:
        lomaps = numpy.load(glob.glob('logodds' + '/' + imname + '/*' + logodds + '_*mean.npz')[0])
        pmean = lomaps['arr_0'].copy()
        pstd = lomaps['arr_1'].copy()
    except:
        print(f'No logodds map for {imname}')
if distype[0] == 'p':
    try:
        probmaps = numpy.load(glob.glob('probmaps' + '/' + imname + '/*' + target + '_*mean.npz')[0])
        pmean = probmaps['arr_0'].copy()
        pstd = probmaps['arr_1'].copy()
    except:
        print(f'No probmap for {imname}')
if not pstd is None:
    pmm = numpy.max(numpy.abs(pmean))
    pmean[0,0] = -pmm
    pmean[-1,-1] = pmm

In [ ]:
# show original image
im = imfunc.read_image('data/jpeg-melanoma-1024x1024/test/' + imname + '.jpg')
plt.figure(figsize=figsize)
ax11 = plt.subplot(1, 2, 1)
pm = plt.imshow(im)

# show mean, median
print('mean')
ax12 = plt.subplot(1, 2, 2)
pm = scaledimshow(pmean * (pstd < 1) * (pmean > 0.1), ax12)

In [ ]:
# mix image
lut = [[32,32,255],[32,255,128]]
mim = imfunc.image_overlay(im, pmean+1.5, lut, max_thresh=numpy.max(pmean+2.5), alpha=-1.0)
plt.figure(figsize=(16,8))
ax11 = plt.subplot(1,2,1)
pm = plt.imshow(im)
ax12 = plt.subplot(1,2,2)
pm = plt.imshow(mim)